<a href="https://colab.research.google.com/github/m-r-tanha/RCNN-Anomaly-detection/blob/master/RCNN_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, normalize, minmax_scale
from tensorflow.contrib import rnn
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
import numpy as np
from google.colab import files
import io
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Wait for 8 seconds...
TensorBoard link:
https://4ecf94bc.ngrok.io


Saving 1000_Sample2.xlsx to 1000_Sample2.xlsx
User uploaded file "1000_Sample2.xlsx" with length 235213 bytes


In [0]:
writer = pd.ExcelWriter('Yexcel.xlsx')
data_path="1000_Sample2.xlsx"
data=pd.read_excel(data_path,sheet_name='Sheet3')
data=data.fillna(0.0)
data.rename(index=data.Cell,inplace=True)
data.drop('Cell',axis=1,inplace=True)
data.Lable_target=data.Lable_target.replace(to_replace=['N','DS','SI','GD','SD','GI'],value=[1,2,3,4,5,6])
X=data.drop('Lable_target',axis=1).values
Y=data.Lable_target
#print(Y)
#Y.describe()


In [0]:
print(X)

[[3.1600e+03 2.9400e+03 2.5300e+03 ... 4.0000e+03 4.0800e+03 3.1200e+03]
 [1.0000e+00 1.0000e+00 1.0000e+00 ... 1.2089e+02 1.2081e+02 6.6960e+01]
 [1.7940e+01 1.8560e+01 1.1290e+01 ... 1.3840e+01 1.3120e+01 8.5600e+00]
 ...
 [2.4770e+01 1.7700e+01 1.6510e+01 ... 1.7420e+01 1.6630e+01 1.8010e+01]
 [1.5300e+00 1.8400e+00 1.4000e+00 ... 2.4000e-01 2.8000e-01 3.1000e-01]
 [4.7200e+00 2.5100e+00 3.0800e+00 ... 5.0500e+00 6.4800e+00 5.9800e+00]]


In [6]:
# creating a noise with the same dimension as the dataset 
mu, sigma = .75, 0.1 
noise = np.random.normal(mu, sigma, [1000,31]) 
noise.shape
#-------------

wn = np.random.randn(len(X),31)
data_wn=X+wn

X=np.append (X,data_wn,axis=0)
Y=np.append(Y,Y,axis=0)
print(X)

[[ 3.16000000e+03  2.94000000e+03  2.53000000e+03 ...  4.00000000e+03
   4.08000000e+03  3.12000000e+03]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  1.20890000e+02
   1.20810000e+02  6.69600000e+01]
 [ 1.79400000e+01  1.85600000e+01  1.12900000e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 2.64238070e+01  1.75011617e+01  1.55326041e+01 ...  1.87181984e+01
   1.71298591e+01  1.89705630e+01]
 [ 7.47556802e-01  2.71514128e-01  6.48289901e-01 ...  9.07078129e-01
   2.56023517e-01 -2.78646657e-01]
 [ 3.16108431e+00  2.08930694e+00  3.32417547e+00 ...  4.67366973e+00
   6.57715614e+00  5.97521840e+00]]


In [0]:
Normalized_X=scale(X, axis=1)
Mi=Normalized_X.min()   
Normalized_X=(-1*Mi)+Normalized_X


X_train=Normalized_X[0:1400]
X_test=Normalized_X[1401:1999]

Y_train=Y[0:1400]
Y_test=Y[1401:1999]

In [8]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

Y_train = np.delete(Y_train , 0, 1)
Y_test = np.delete(Y_test , 0, 1)
# Training Parametersznum_steps = 30
num_steps=700
batch_size = 120
display_step = 1
strides = 1
k = 1

# Network Parameters
num_input = 31  #  data input (img shape: 28*28)
num_hidden = 100
num_classes = 6  #  total classes (0-9 digits)
dropout = 0.7  # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)  # dropout (keep probability)

is_training = tf.placeholder(tf.bool, name='MODE')
#is_training=True

# Batch Normal
def batch_norm_layer(inputT, is_training=True, scope=None):
    # Note: is_training is tf.placeholder(tf.bool) type
    return tf.cond(is_training,
                    lambda: batch_norm(inputT, is_training=True,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope=scope),
                    lambda: batch_norm(inputT, is_training=False,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9,
                    scope=scope, reuse = True))
#
# Store layers weight & bias
# The first three convolutional layer
w_c_1 = tf.Variable(tf.random_normal([1, 3, 1, 28]))
w_c_2 = tf.Variable(tf.random_normal([1, 3, 28, 56]))
w_c_3 = tf.Variable(tf.random_normal([1, 3, 56, 112]))
b_c_1 = tf.Variable(tf.zeros([28]))
b_c_2 = tf.Variable(tf.zeros([56]))
b_c_3 = tf.Variable(tf.zeros([112]))

# The second three convolutional layer weights
w_c_4 = tf.Variable(tf.random_normal([1, 3, 112, 224]))
w_c_5 = tf.Variable(tf.random_normal([1, 3, 224, 448]))
w_c_6 = tf.Variable(tf.random_normal([1, 3, 448, 896]))
b_c_4 = tf.Variable(tf.zeros([224]))
b_c_5 = tf.Variable(tf.zeros([448]))
b_c_6 = tf.Variable(tf.zeros([896]))

# Fully connected weight
w_f_1 = tf.Variable(tf.random_normal([1 * 31 * 896, 1792])) # fully connected, 1*3*896 inputs, 2048 outputs
w_f_2 = tf.Variable(tf.random_normal([1792, 896]))
w_f_3 = tf.Variable(tf.random_normal([896, 448]))
b_f_1 = tf.Variable(tf.zeros([1792]))
b_f_2 = tf.Variable(tf.zeros([896]))
b_f_3 = tf.Variable(tf.zeros([448]))

# output layer weight
w_out = tf.Variable(tf.random_normal([448, num_classes]))
b_out = tf.Variable(tf.zeros([num_classes]))

#
# Define model
x = tf.reshape(X, shape=[-1, 1, 31, 1])

# first layer convolution
conv1 = tf.nn.conv2d(x, w_c_1, strides=[1, 1, 1, 1], padding='SAME') + b_c_1
conv1=batch_norm(conv1, is_training=True,center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope='conv1_bn')
conv1 = tf.nn.relu(conv1)


# second layer convolution
conv2 = tf.nn.conv2d(conv1, w_c_2, strides=[1, strides, strides, 1], padding='SAME') + b_c_2
conv2=batch_norm(conv2, is_training=True,center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope='conv2_bn')
conv2 = tf.nn.relu(conv2)

# third layer convolution
conv3 = tf.nn.conv2d(conv2, w_c_3, strides=[1, strides, strides, 1], padding='SAME') + b_c_3
conv3=batch_norm(conv3, is_training=True,center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope='conv3_bn')
conv3 = tf.nn.relu(conv3)

# first Max Pooling (down-sampling)
pool_1 = tf.nn.max_pool(conv3, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# fourth layer convolution
conv4 = tf.nn.conv2d(pool_1, w_c_4, strides=[1, strides, strides, 1], padding='SAME') + b_c_4
conv4=batch_norm(conv4, is_training=True,center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope='conv4_bn')
conv4 = tf.nn.relu(conv4)
conv4 = tf.nn.dropout(conv4, dropout)


# fifth layer convolution
conv5 = tf.nn.conv2d(conv4, w_c_5, strides=[1, strides, strides, 1], padding='SAME') + b_c_5
conv5=batch_norm(conv5, is_training=True,center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope='conv5_bn')
conv5 = tf.nn.relu(conv5)

# sixth layer convolution
conv6 = tf.nn.conv2d(conv5, w_c_6, strides=[1, strides, strides, 1], padding='SAME') + b_c_6
conv6=batch_norm(conv6, is_training=True,center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope='conv6_bn')
conv6 = tf.nn.relu(conv6)
#conv6 = tf.nn.dropout(conv6, dropout)

# second Max Pooling (down-sampling)
pool_2 = tf.nn.max_pool(conv6, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

pool_2 = tf.reshape(pool_2, [-1,31,896])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden,num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


def RNN(x, weights, biases):

    lstm_cell = rnn.BasicLSTMCell(num_hidden,reuse=tf.AUTO_REUSE)

    outputs, states = tf.nn.dynamic_rnn(lstm_cell, x , dtype=tf.float32)

    return tf.matmul(outputs[:,-1], weights['out']) + biases['out']

logits = RNN(pool_2, weights, biases)
prediction = tf.nn.softmax(logits)

#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$4
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#extra

#rec, rec_op = tf.metrics.recall(labels=tf.argmax(Y, 1), predictions=tf.argmax(prediction, 1))
#pre, pre_op = tf.metrics.precision(labels=tf.argmax(Y, 1), predictions=tf.argmax(prediction, 1))
#rec pre

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Please don't change these.
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Tensorboard
writer=tf.summary.FileWriter('./graphs', tf.get_default_graph())
#tbc.save_image(title="test_title")

writer.close()
A=pd.DataFrame({'loss':[], 'Acc':[]})

# Start training
with tf.Session(config=config) as sess:
    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps + 1):

        sess.run(train_op, feed_dict={X: X_train, Y: Y_train, keep_prob: 0.7})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc= sess.run([loss_op, accuracy], feed_dict={X: X_train,
                                                                 Y: Y_train,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) )
            df=pd.DataFrame({'loss':[loss], 'Acc':[acc]})
            A=A.append(df) 
    saver=tf.train.Saver()
    save_path=saver.save(sess,"./TF_ModelwBN_for_Paper4.ckpt")
    tbc.save_value("graph_name", "line_name", num_steps, loss)
    #tbc.flush_line(line_name)
    tbc.close()
    print("Optimization Finished!")    
    print("Testing Accuracy:", \
          sess.run(accuracy, feed_dict={X: X_test , Y: Y_test}))


Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Step 1, Minibatch Loss= 7.5704, Training Accuracy= 0.249
Step 2, Minibatch Loss= 6.4777, Training Accuracy= 0.276
Step 3, Minibatch Loss= 4.0126, Training Accuracy= 0.249
Step 4, Minibatch Loss= 1.9351, Trai

In [0]:
print('done saving at',save_path)

In [0]:
files.download("TF_ModelwBN_for_Paper4.ckpt.meta")
files.download("TF_ModelwBN_for_Paper4.ckpt.index")
files.download("TF_ModelwBN_for_Paper4.ckpt.data-00000-of-00001")